In [1]:
!git clone https://github.com/minhloc29/Japanese-Characters-Recognition.git

Cloning into 'Japanese-Characters-Recognition'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 117 (delta 49), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 16.82 MiB | 12.94 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [5]:
%cd Japanese-Characters-Recognition
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import albumentations as A
from data_loader import JapaneseDataset
from unet import resnet_unet
from losses_and_metrics import dice_coef, iou_score, bce_dice_loss

/kaggle/working/Japanese-Characters-Recognition


In [ ]:
train = pd.read_csv("/kaggle/input/train-images/train.csv")
labels = train['labels'].to_list()
root_images_dir = "/kaggle/input/train-images/train_images/train_images"
relative_image_urls = os.listdir(root_images_dir)
image_urls = [os.path.join(root_images_dir, relative_image_url) for relative_image_url in relative_image_urls]

count_labels = [len(np.array(label.split()).reshape(-1, 5)) for label in labels]
range_counts = np.array([0, 50, 100, 150, 250, 300, 400, 500, 700])
range_labels = []

for count_char_label in count_labels:
    for index, range_count in enumerate(range_counts[:-1]):
        if count_char_label >= range_count and count_char_label <= range_counts[index + 1]:
            range_labels.append(index)
            break

train_img_urls, val_img_urls, train_labels, val_labels = \
        train_test_split(image_urls, labels, test_size=0.1, stratify=range_labels, random_state=42)

train_generator = JapaneseDataset(train_img_urls, train_labels, batch_size=8, augment=True, shuffle = True)
evaluator_generator = JapaneseDataset(val_img_urls, val_labels, batch_size=8, augment=False, shuffle = False)
train = train_generator.create_tf_dataset()
test = evaluator_generator.create_tf_dataset()

In [8]:
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

csv = CSVLogger(
    filename = "/kaggle/working/Japanese-Characters-Recognition/models/training_log.csv", append = True
)
checkpoint = ModelCheckpoint(
    filepath = "model.keras",
    monitor = 'val_loss',
    verbose = 1,
    mode = 'min',
    save_best_only = True,
    save_weights_only = False,
    save_freq = 'epoch',
    initial_value_threshold = 0.9
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2)

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
from tensorflow.keras import mixed_precision

# Set the mixed precision policy to 'mixed_bfloat16'
policy = mixed_precision.Policy('mixed_bfloat16')
mixed_precision.set_global_policy(policy)

Num GPUs Available:  2


In [13]:
#Using 2 GPUs in Kaggle GPU T4x2 to accelerate training

strategy = tf.distribute.MirroredStrategy()
tf.debugging.set_log_device_placement(True)

with strategy.scope():
    model = resnet_unet(start_neurons = 16)
    optim = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss=bce_dice_loss, optimizer=optim, metrics=[iou_score, dice_coef])

history = model.fit(train, epochs=10, validation_data=test, 
                   steps_per_epoch = 203, validation_steps=23, callbacks = [csv, checkpoint, reduce_lr])


Epoch 1/10


2024-12-10 05:57:52.457606: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_1068/cond/StatefulPartitionedCall/functional_5_1/dropout_12_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - dice_coef: 0.7087 - iou_score: 0.5629 - loss: 0.6412
Epoch 1: val_loss improved from 0.90000 to 0.50688, saving model to model.keras
203/203 ━━━━━━━━━━━━━━━━━━━━ 348s 1s/step - dice_coef: 0.7090 - iou_score: 0.5632 - loss: 0.6403 - val_dice_coef: 0.7437 - val_iou_score: 0.5944 - val_loss: 0.5069 - learning_rate: 0.0100
Epoch 2/10
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - dice_coef: 0.8248 - iou_score: 0.7028 - loss: 0.3229
Epoch 2: val_loss improved from 0.50688 to 0.33093, saving model to model.keras
203/203 ━━━━━━━━━━━━━━━━━━━━ 272s 1s/step - dice_coef: 0.8248 - iou_score: 0.7028 - loss: 0.3229 - val_dice_coef: 0.8128 - val_iou_score: 0.6852 - val_loss: 0.3309 - learning_rate: 0.0100


In [9]:
#This code is used for further training later

strategy = tf.distribute.MirroredStrategy()
tf.debugging.set_log_device_placement(True)

with strategy.scope():
    model = tf.keras.models.load_model('/kaggle/working/Japanese-Characters-Recognition/models/model.keras',
                                   custom_objects = {"dice_coef": dice_coef, 
                                                     "iou_score": iou_score,
                                                     "bce_dice_loss": bce_dice_loss})
    
    optim = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss=bce_dice_loss, optimizer=optim, metrics=[iou_score, dice_coef])

history = model.fit(train, epochs=5, validation_data=test, 
                   steps_per_epoch = 203, validation_steps=23, callbacks = [csv, checkpoint, reduce_lr])


Epoch 1/5


2024-12-11 07:28:13.528219: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_1068/cond/StatefulPartitionedCall/functional_3_1/dropout_6_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - dice_coef: 0.8863 - iou_score: 0.7963 - loss: 0.2102
Epoch 1: val_loss improved from 0.90000 to 0.19891, saving model to model.keras
203/203 ━━━━━━━━━━━━━━━━━━━━ 364s 1s/step - dice_coef: 0.8863 - iou_score: 0.7963 - loss: 0.2101 - val_dice_coef: 0.8878 - val_iou_score: 0.7990 - val_loss: 0.1989 - learning_rate: 0.0010
Epoch 2/5
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - dice_coef: 0.8931 - iou_score: 0.8072 - loss: 0.1989
Epoch 2: val_loss improved from 0.19891 to 0.17624, saving model to model.keras
203/203 ━━━━━━━━━━━━━━━━━━━━ 276s 1s/step - dice_coef: 0.8931 - iou_score: 0.8072 - loss: 0.1989 - val_dice_coef: 0.9051 - val_iou_score: 0.8268 - val_loss: 0.1762 - learning_rate: 0.0010
Epoch 3/5
203/203 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - dice_coef: 0.8968 - iou_score: 0.8133 - loss: 0.1906
Epoch 3: val_loss improved from 0.17624 to 0.17290, saving model to model.keras
203/203 ━━━━━━━━━━━━━━━━━━━━ 273s 1s/step - dice_coef: 0.8968 - iou_score: 0.81